# Coursera Capstone Project
This notebook is going to be mainly used for the capstone project.

In [ ]:
import pandas as pd
import numpy as np
import requests
from bs4 import BeautifulSoup

#### The first step is to scrape a wikipedia page to get the name of all neighboorhoods in Toronto.

In [ ]:
url = "https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"
r = requests.get(url)
html = r.text

#### We are going to use the Beautifulsoup library to pull the data out of thw Html file that we got from wikipedia

In [ ]:
soup = BeautifulSoup(html,'lxml')
table = soup.find_all('tr')##get all the items that are in the table
table = table[0:289] ##get only the relevant lines
##Separate the items in the table
postcode = []
borough = []
neigh = []
for i in table:
    words = i.text.split('\n')
    postcode.append(words[1])
    borough.append(words[2])
    neigh.append(words[3])

#### Let's create a dataframe with the items that we got

In [ ]:
df_toronto = pd.DataFrame({'PostalCode':postcode,'Borough':borough,'Neighboorhood':neigh})
df_toronto = df_toronto.drop(0)
df_toronto.head()

#### Next, we wil remove all lines that has Borough = Not assigned

In [ ]:
df_toronto = df_toronto[df_toronto['Borough'] != 'Not assigned']
df_toronto.head()

#### If a cell has a borough but a Not assigned neighborhood, then the neighborhood will be the same as the borough.

In [ ]:
for index,row in df_toronto.iterrows():
    if(row['Neighboorhood']=='Not assigned'):
        df_toronto.loc[index,'Neighboorhood'] = df_toronto.loc[index,'Borough']
df_toronto = df_toronto.reset_index().drop(columns="index") ##reset index
df_toronto.head()

#### Note that more than one Neighboorhood can exist in one postal code area. In this cases, we will combine these rows. 

In [ ]:
def merge_rows(data,postal):
    df = data[data["PostalCode"] == postal]
    neigh = ''
    for i , row in df.iterrows():
        borough = row['Borough']
        neigh += row['Neighboorhood'] + ', '
    return postal,borough,neigh[:-2]

In [97]:
postalcodes = df_toronto['PostalCode'].unique()
data_toronto = pd.DataFrame(columns=['PostalCode','Borough','Neighboorhood'])
for p in postalcodes:
    postal,borough,neigh = merge_rows(df_toronto,p)
    data_toronto.loc[postal] = [postal,borough,neigh]
data_toronto = data_toronto.reset_index().drop(columns='index')
data_toronto

,PostalCode,Borough,Neighboorhood
M3A,M3A,North York,Parkwoods
M4A,M4A,North York,Victoria Village
M5A,M5A,Downtown Toronto,"Harbourfront, Regent Park"
M6A,M6A,North York,"Lawrence Heights, Lawrence Manor"
M7A,M7A,Queen's Park,Queen's Park
